In [1]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv  # Importing GCN layer

path = './ppi_data/'
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


class GCNPPI(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(train_dataset.num_features, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, train_dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


times = []
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_f1:.4f}, '
          f'Test: {test_f1:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")


Epoch: 001, Loss: 0.6069, Val: 0.4294, Test: 0.4304
Epoch: 002, Loss: 0.5827, Val: 0.4022, Test: 0.4052
Epoch: 003, Loss: 0.5705, Val: 0.4596, Test: 0.4669
Epoch: 004, Loss: 0.5545, Val: 0.5031, Test: 0.5130
Epoch: 005, Loss: 0.5462, Val: 0.5095, Test: 0.5184
Epoch: 006, Loss: 0.5446, Val: 0.4912, Test: 0.4995
Epoch: 007, Loss: 0.5390, Val: 0.4742, Test: 0.4815
Epoch: 008, Loss: 0.5306, Val: 0.5135, Test: 0.5230
Epoch: 009, Loss: 0.5246, Val: 0.4813, Test: 0.4901
Epoch: 010, Loss: 0.5207, Val: 0.5024, Test: 0.5132
Epoch: 011, Loss: 0.5159, Val: 0.5174, Test: 0.5301
Epoch: 012, Loss: 0.5155, Val: 0.5312, Test: 0.5458
Epoch: 013, Loss: 0.5115, Val: 0.5185, Test: 0.5306
Epoch: 014, Loss: 0.5094, Val: 0.4798, Test: 0.4948
Epoch: 015, Loss: 0.5036, Val: 0.5073, Test: 0.5211
Epoch: 016, Loss: 0.5010, Val: 0.5099, Test: 0.5205
Epoch: 017, Loss: 0.4974, Val: 0.5266, Test: 0.5376
Epoch: 018, Loss: 0.4953, Val: 0.5183, Test: 0.5307
Epoch: 019, Loss: 0.4924, Val: 0.5461, Test: 0.5583
Epoch: 020, 